In [27]:
# dependencies
import pandas as pd
import json
import ast
from datetime import datetime
import scipy.stats as st
import numpy as np

### Reading in data
<p>The <a href="https://www.kaggle.com/rounakbanik/the-movies-dataset?select=credits.csv">dataset</a> was obtained from Kaggle and consists of the following files:<p>

* **movies_metadata.csv**: The main Movies Metadata file. Contains information on 45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, revenue, release dates, languages, production countries and companies.

* **keywords.csv**: Contains the movie plot keywords for our MovieLens movies. Available in the form of a stringified JSON Object.

* **credits.csv**: Consists of Cast and Crew Information for all our movies. Available in the form of a stringified JSON Object.

* **links.csv**: The file that contains the TMDB and IMDB IDs of all the movies featured in the Full MovieLens dataset.

* **links_small.csv**: Contains the TMDB and IMDB IDs of a small subset of 9,000 movies of the Full Dataset.

* **ratings_small.csv**: The subset of 100,000 ratings from 700 users on 9,000 movies.

# Creating Production Table

### Exploring data

In [28]:
credits = pd.read_csv("Data/credits.csv")
keywords = pd.read_csv("Data/keywords.csv")
movies_meta = pd.read_csv("Data/movies_metadata.csv", low_memory=False)
ratings = pd.read_csv("Data/ratings.csv")
ratings_small = pd.read_csv("Data/ratings_small.csv")

In [29]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [30]:
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [31]:
movies_meta.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [32]:
# Review all available columns in movies metadata table
print(f"Number of columns:\t{len(movies_meta.columns)}")
print("Column names:", end="\t\t")
for column in movies_meta.columns:
    print(column, end=", ")

Number of columns:	24
Column names:		adult, belongs_to_collection, budget, genres, homepage, id, imdb_id, original_language, original_title, overview, popularity, poster_path, production_companies, production_countries, release_date, revenue, runtime, spoken_languages, status, tagline, title, video, vote_average, vote_count, 

In [33]:
movies_meta.iloc[:,:11].head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519


In [34]:
movies_meta.iloc[:,12:].head()

,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [35]:
# See number of records in ratings data table
print(f"Total number of ratings records:\t {ratings.shape[0]}")

# View ratings table
ratings.head()

Total number of ratings records:	 26024289


,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


### Transforming data (convert ratings timestamp, production)

In [36]:
# Convert ratings timestamp to date

In [37]:
# Create custom function to convert time data
def convert_tstamp_to_date(x):
    y = datetime.fromtimestamp(x).strftime('%Y-%m-%d')
    return y

# Test if the function works
convert_tstamp_to_date(ratings['timestamp'][0])

'2015-03-09'

In [38]:
ratings['date']=ratings['timestamp'].apply(convert_tstamp_to_date)
ratings.head()

,userId,movieId,rating,timestamp,date
0,1,110,1.0,1425941529,2015-03-09
1,1,147,4.5,1425942435,2015-03-09
2,1,858,5.0,1425941523,2015-03-09
3,1,1221,5.0,1425941546,2015-03-09
4,1,1246,5.0,1425941556,2015-03-09


In [39]:
# From the movie_meta df select and transform only columns 
# that include json format nestedentries

movie_meta_transformed = []

for i, row in movies_meta.iterrows():
    #     print(f"Processing ---  Row {i}")
    
    # Parsing collections data    
    coll_data = row['belongs_to_collection']
    try :
        coll_id   = json.loads(coll_data.replace("'",'"'))['id']
        coll_name = json.loads(coll_data.replace("'",'"'))['name']
        coll_poster = json.loads(coll_data.replace("'",'"'))['poster_path']
    except :
        coll_id=row['belongs_to_collection']
        coll_name=row['belongs_to_collection']
        coll_poster=row['belongs_to_collection']
    
    # Parsing production companies
    prod_comp_data = row['production_companies']
    
    try :
        prod_comp_name = []
        prod_comp_name_str = ""
        counter=0
        for comp_name in json.loads(prod_comp_data.replace("'",'"')):
            name=comp_name['name']
            prod_comp_name.append(name)
            if counter == 0:
                prod_comp_name_str += f"{name}"
                counter += 1
            else :
                prod_comp_name_str += f", {name}"
    except:
        prod_comp_name_str = row['production_companies']
    
    # Parsing production countries
    prod_count_data = row['production_countries']
    
    try :
        prod_count_name = []
        prod_count_name_str = ""
        counter=0
        for count_name in json.loads(prod_count_data.replace("'",'"')):
            name = count_name['name']
            abb  = count_name['iso_3166_1']
            prod_count_name.append(name)
            if counter == 0:
                prod_count_name_str += f"{name}"
                counter += 1
            else :
                prod_count_name_str += f", {name}"
    except :
        prod_count_name_str = row['production_countries']
        abb = ""
    
    # Pull row data into dictionary
    movies_dic={
        "movieId":           row['id'],
        "collectionId":      coll_id,
        "collectionName":    coll_name,
        'productionCompanies':
                             prod_comp_name_str,
        'productionCountries':
                             prod_count_name_str, 
        'countryAbb':        abb,
        # additionally parse out release year
        'releaseYear':       str(row['release_date']).split('-')[0]
    }
    
    movie_meta_transformed.append(movies_dic)

movie_meta_transformed = pd.DataFrame(movie_meta_transformed)
movie_meta_transformed.head()

,movieId,collectionId,collectionName,productionCompanies,productionCountries,countryAbb,releaseYear
0,862,10194,Toy Story Collection,Pixar Animation Studios,United States of America,US,1995
1,8844,NaN,NaN,"TriStar Pictures, Teitler Film, Interscope Com...",United States of America,US,1995
2,15602,119050,Grumpy Old Men Collection,"Warner Bros., Lancaster Gate",United States of America,US,1995
3,31357,NaN,NaN,Twentieth Century Fox Film Corporation,United States of America,US,1995
4,11862,96871,Father of the Bride Collection,"Sandollar Productions, Touchstone Pictures",United States of America,US,1995


In [40]:
pd.DataFrame(movie_meta_transformed.describe().loc['unique'])

,unique
movieId,45436
collectionId,1698
collectionName,1698
productionCompanies,22672
productionCountries,2393
countryAbb,140
releaseYear,139


In [41]:
# Extract all possible genres
all_genres=[]
for i, row in movies_meta.iterrows():
    #     print(f"Processing ---  Row {i}")
    
    # Parsing genres data
    genres_data = row['genres']
    
    try :
        for genre in json.loads(genres_data.replace("'",'"')):
            name = genre['name']
            
            if name not in all_genres:
                all_genres.append(name)
    except :
        print("hi")

len(all_genres)

32

In [42]:
# Extract all possible collections
all_collections=[]
for i, row in movies_meta.iterrows():
    #     print(f"Processing ---  Row {i}")
    
    # Parsing genres data
    coll_data = row['belongs_to_collection']
    
    try :
        name = json.loads(coll_data.replace("'",'"'))['name']
        if name not in all_collections:
            all_collections.append(name)
    except :
        all_collections = all_collections

len(all_collections)

1078

# Creating Budget/Revenue Table

In [43]:
# budget and revenue data
data = pd.read_csv('Data/movies_metadata.csv')
data = pd.DataFrame(data)
data = data[['id','title','budget','revenue']]
data.head()

C:\Users\Doc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,title,budget,revenue
0,862,Toy Story,30000000,373554033.0
1,8844,Jumanji,65000000,262797249.0
2,15602,Grumpier Old Men,0,0.0
3,31357,Waiting to Exhale,16000000,81452156.0
4,11862,Father of the Bride Part II,0,76578911.0


# Creating Rating Table

In [44]:
# ratings file
rating = pd.read_csv('Data/ratings.csv')
rating.head()


,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [45]:
rating_df = pd.DataFrame(rating)
rating_df = rating[['movieId','rating']]
rating_df.head()

,movieId,rating
0,110,1.0
1,147,4.5
2,858,5.0
3,1221,5.0
4,1246,5.0


In [46]:
rating_df.rename(columns={'movieId':'id'}, inplace=True,)
rating_df.head()

C:\Users\Doc\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,id,rating
0,110,1.0
1,147,4.5
2,858,5.0
3,1221,5.0
4,1246,5.0


In [47]:
# Average rating by movie id
rating_df = rating_df.groupby('id')['rating'].mean().reset_index()
rating_df

,id,rating
0,1,3.888157
1,2,3.236953
2,3,3.175550
3,4,2.875713
4,5,3.079565
...,...,...
45110,176267,4.000000
45111,176269,3.500000
45112,176271,5.000000
45113,176273,1.000000


In [48]:
# movies dataset
movies = pd.read_csv('Data/movies_metadata.csv')
movies = pd.DataFrame(movies)
movies = movies[['id','title','budget','revenue']]
movies.head()


C:\Users\Doc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,title,budget,revenue
0,862,Toy Story,30000000,373554033.0
1,8844,Jumanji,65000000,262797249.0
2,15602,Grumpier Old Men,0,0.0
3,31357,Waiting to Exhale,16000000,81452156.0
4,11862,Father of the Bride Part II,0,76578911.0


In [49]:
# include a column to calculate the profit for each movie
movies["profit"] = ""
movies

,id,title,budget,revenue,profit
0,862,Toy Story,30000000,373554033.0,
1,8844,Jumanji,65000000,262797249.0,
2,15602,Grumpier Old Men,0,0.0,
3,31357,Waiting to Exhale,16000000,81452156.0,
4,11862,Father of the Bride Part II,0,76578911.0,
...,...,...,...,...,...
45461,439050,Subdue,0,0.0,
45462,111109,Century of Birthing,0,0.0,
45463,67758,Betrayal,0,0.0,
45464,227506,Satan Triumphant,0,0.0,


In [50]:
# cleaning the budget column
clean_movies = movies[movies["budget"] != "/ff9qCepilowshEtG2GYWwzt2bs4.jpg"]
clean_movies = clean_movies[clean_movies["budget"] != "/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg"]
clean_movies = clean_movies[clean_movies["budget"] != "/zaSf5OG7V8X8gqFvly88zDdRm46.jpg"]
clean_movies

,id,title,budget,revenue,profit
0,862,Toy Story,30000000,373554033.0,
1,8844,Jumanji,65000000,262797249.0,
2,15602,Grumpier Old Men,0,0.0,
3,31357,Waiting to Exhale,16000000,81452156.0,
4,11862,Father of the Bride Part II,0,76578911.0,
...,...,...,...,...,...
45461,439050,Subdue,0,0.0,
45462,111109,Century of Birthing,0,0.0,
45463,67758,Betrayal,0,0.0,
45464,227506,Satan Triumphant,0,0.0,


In [51]:
# Budget and revenue as float
clean_movies['budget'] = clean_movies['budget'].astype(float)
clean_movies['revenue'] = clean_movies['revenue'].astype(float)
clean_movies

,id,title,budget,revenue,profit
0,862,Toy Story,30000000.0,373554033.0,
1,8844,Jumanji,65000000.0,262797249.0,
2,15602,Grumpier Old Men,0.0,0.0,
3,31357,Waiting to Exhale,16000000.0,81452156.0,
4,11862,Father of the Bride Part II,0.0,76578911.0,
...,...,...,...,...,...
45461,439050,Subdue,0.0,0.0,
45462,111109,Century of Birthing,0.0,0.0,
45463,67758,Betrayal,0.0,0.0,
45464,227506,Satan Triumphant,0.0,0.0,


In [52]:
# obtain the profit as the difference between revenue and budget
clean_movies['profit'] = clean_movies['revenue'] - clean_movies['budget']
clean_movies

,id,title,budget,revenue,profit
0,862,Toy Story,30000000.0,373554033.0,343554033.0
1,8844,Jumanji,65000000.0,262797249.0,197797249.0
2,15602,Grumpier Old Men,0.0,0.0,0.0
3,31357,Waiting to Exhale,16000000.0,81452156.0,65452156.0
4,11862,Father of the Bride Part II,0.0,76578911.0,76578911.0
...,...,...,...,...,...
45461,439050,Subdue,0.0,0.0,0.0
45462,111109,Century of Birthing,0.0,0.0,0.0
45463,67758,Betrayal,0.0,0.0,0.0
45464,227506,Satan Triumphant,0.0,0.0,0.0


In [53]:
# rating_df

In [54]:
# Identify the type
clean_movies.dtypes,rating_df.dtypes

(id          object
 title       object
 budget     float64
 revenue    float64
 profit     float64
 dtype: object,
 id          int64
 rating    float64
 dtype: object)

In [55]:
# converting to the same type of variable
clean_movies['id'] = clean_movies['id'].astype('int64')

In [56]:
#same types
clean_movies.dtypes,rating_df.dtypes

(id           int64
 title       object
 budget     float64
 revenue    float64
 profit     float64
 dtype: object,
 id          int64
 rating    float64
 dtype: object)

In [57]:
# merge both cleanig dataframes

movie_rating = clean_movies.merge(rating_df, on='id', how='left')
movie_rating

,id,title,budget,revenue,profit,rating
0,862,Toy Story,30000000.0,373554033.0,343554033.0,3.598930
1,8844,Jumanji,65000000.0,262797249.0,197797249.0,3.760163
2,15602,Grumpier Old Men,0.0,0.0,0.0,NaN
3,31357,Waiting to Exhale,16000000.0,81452156.0,65452156.0,NaN
4,11862,Father of the Bride Part II,0.0,76578911.0,76578911.0,NaN
...,...,...,...,...,...,...
45458,439050,Subdue,0.0,0.0,0.0,NaN
45459,111109,Century of Birthing,0.0,0.0,0.0,2.583333
45460,67758,Betrayal,0.0,0.0,0.0,NaN
45461,227506,Satan Triumphant,0.0,0.0,0.0,NaN


In [58]:
print(ratings.count(), ratings['userId'].nunique())

userId       26024289
movieId      26024289
rating       26024289
timestamp    26024289
date         26024289
dtype: int64 270896


# Creating Genres and Profit Tables

In [59]:
# Reference to CSV and reading CSV into Pandas DataFrame
csv_path1 = "Data/credits.csv"
csv_path2 = "Data/keywords.csv"
csv_path3 = "Data/links.csv"
csv_path4 = "Data/links_small.csv"
csv_path5 = "Data/movies_metadata.csv"
csv_path6 = "Data/ratings.csv"
csv_path7 = "Data/ratings_small.csv"


# merging final combined CSV meta file


credits_df = pd.read_csv(csv_path1)
keywords_df = pd.read_csv(csv_path2)
links_df = pd.read_csv(csv_path3)
links_small_df = pd.read_csv(csv_path4)
movies_metadata_df = pd.read_csv(csv_path5)
ratings_df = pd.read_csv(csv_path6)
ratings_small_df = pd.read_csv(csv_path7)


C:\Users\Doc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [60]:
movie_genres =movies_metadata_df["genres"]
movie_genres.head()


0    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1    [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2    [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                       [{'id': 35, 'name': 'Comedy'}]
Name: genres, dtype: object

In [61]:
def extract_names(list_genres):
    list_genres = json.loads(list_genres.replace("'",'"'))

    genres_names = []
    for item in list_genres:
        
        genres_names.append(item['name'])
    return ",".join(genres_names)

In [62]:
extract_names(movie_genres[0])

'Animation,Comedy,Family'

In [63]:
movie_genres.apply(extract_names)

0         Animation,Comedy,Family
1        Adventure,Fantasy,Family
2                  Romance,Comedy
3            Comedy,Drama,Romance
4                          Comedy
                   ...           
45461                Drama,Family
45462                       Drama
45463       Action,Drama,Thriller
45464                            
45465                            
Name: genres, Length: 45466, dtype: object

In [64]:
#test = '[{"id": 16, "name": "Animation"}]'
test=movie_genres[0].replace("'",'"')
json.loads(test)


[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [65]:
movies_metadata_df["genres"] =movie_genres.apply(extract_names)
movies_metadata_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"Animation,Comedy,Family",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"Adventure,Fantasy,Family",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"Romance,Comedy",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"Comedy,Drama,Romance",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,Comedy,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [66]:
movie_vote_df = movies_metadata_df[['id','title','genres','vote_average']]
movie_vote_df


,id,title,genres,vote_average
0,862,Toy Story,"Animation,Comedy,Family",7.7
1,8844,Jumanji,"Adventure,Fantasy,Family",6.9
2,15602,Grumpier Old Men,"Romance,Comedy",6.5
3,31357,Waiting to Exhale,"Comedy,Drama,Romance",6.1
4,11862,Father of the Bride Part II,Comedy,5.7
...,...,...,...,...
45461,439050,Subdue,"Drama,Family",4.0
45462,111109,Century of Birthing,Drama,9.0
45463,67758,Betrayal,"Action,Drama,Thriller",3.8
45464,227506,Satan Triumphant,,0.0


In [83]:
movie_vote_df.to_csv("movie_genres.csv")

# Creating keyword table

In [67]:
movies = pd.read_csv('Data\movies_metadata.csv')
movies = pd.DataFrame(movies)
movies = movies[['id','title','overview','release_date']]
movies.head()

C:\Users\Doc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,title,overview,release_date
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",1995-10-30
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,1995-12-15
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,1995-12-22
3,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",1995-12-22
4,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,1995-02-10


In [79]:
keywords = pd.read_csv('Data\keywords.csv')
keywords = pd.DataFrame(keywords)
keywords = keywords[['id','keywords']]
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [82]:
movie_keywords = []

keys = keywords['keywords']
movie_id = list(keywords['id'])

for key in keys:

    words = key
    words = ast.literal_eval(words)
        
    try:
        words1 = words[0]['name']
    except:
        words1 = 'null'
        
    try:
        words2 = words[1]['name']
    except:
        words2 = 'null'
        
    try:
        words3 = words[2]['name']
    except:
        words3 = 'null'
        
    try:
        words4 = words[3]['name']
    except:
        words4 = 'null'

    try:
        words5 = words[4]['name']
    except:
        words5 = 'null'
    
    keys_list = f"{words1}, {words2}, {words3}, {words4}, {words5}" 
    movie_keywords.append(keys_list)

keywords_df = {
    'movieid': movie_id,
    'keywords': movie_keywords
}

keywords_df = pd.DataFrame(keywords_df)

keywords_df.head()

,movieid,keywords
0,862,"jealousy, toy, boy, friendship, friends"
1,8844,"board game, disappearance, based on children's..."
2,15602,"fishing, best friend, duringcreditsstinger, ol..."
3,31357,"based on novel, interracial relationship, sing..."
4,11862,"baby, midlife crisis, confidence, aging, daughter"


In [81]:
keywords_df.to_csv("keywords.csv")

# Exporting to PostgreSQL

In [ ]:
# Import sqlalchemy engine
from sqlalchemy import create_engine

In [ ]:
# setup connection
rds_connection_string = "postgres:postgres@localhost:5432/movielens"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
#  review table names
engine.table_names()

In [ ]:
movie_rating_UD = pd.read_csv("movie_rating_UD.csv")
movie_rating_UD_2=movie_rating_UD.drop_duplicates(subset='movieid')
movie_rating_UD_2.to_sql(name='movie_rating', con=engine, if_exists='append', index=False)

In [ ]:
movie_keywords = pd.read_csv("keywords.csv")
movie_keywords_2=movie_keywords[['movieid','keywords']].drop_duplicates(subset='movieid').rename(columns={'Unnamed: 0':'index'})
movie_keywords_2.to_sql(name='keywords', con=engine, if_exists='append', index=False)

In [70]:
movie_genres = pd.read_csv("movie_genres.csv")
movie_genres_2=movie_genres[['id','genres','vote_average']].rename(columns={'id':'movieid'})
movie_genres_3=movie_genres_2.drop_duplicates(subset='movieid')
movie_genres_4=movie_genres_3[~movie_genres_3['movieid'].str.contains('-')]
movie_genres_4.to_sql(name='movie_genres', con=engine, if_exists='append', index=False)

NameError: name 'engine' is not defined